# Training your First Model in Python

<img align="left" width="130" src="https://raw.githubusercontent.com/PacktPublishing/Amazon-SageMaker-Cookbook/master/Extra/cover-small-padded.png"/>

This notebook contains the code to help readers work through one of the recipes of the book [Machine Learning with Amazon SageMaker Cookbook: 80 proven recipes for data scientists and developers to perform ML experiments and deployments](https://www.amazon.com/Machine-Learning-Amazon-SageMaker-Cookbook/dp/1800567030)

### How to do it...

In [ ]:
%store -r df_all_data

In [ ]:
from sklearn.model_selection import train_test_split

X = df_all_data['management_experience_months'].values 
y = df_all_data['monthly_salary'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
import pandas as pd
df_training_data = pd.DataFrame({ 'monthly_salary': y_train, 'management_experience_months': X_train})
df_training_data

In [ ]:
!mkdir -p tmp

In [ ]:
df_training_data.to_csv('tmp/training_data.csv', header=False, index=False)

In [ ]:
s3_bucket = '<insert bucket name here>'
prefix = 'chapter01'

In [ ]:
!aws s3 cp tmp/training_data.csv s3://{s3_bucket}/{prefix}/input/training_data.csv

In [ ]:
import sagemaker 
import boto3
from sagemaker import get_execution_role 

role = get_execution_role()
session = sagemaker.Session()
region_name = boto3.Session().region_name

In [ ]:
training_s3_input_location = f"s3://{s3_bucket}/{prefix}/input/training_data.csv" 
training_s3_output_location = f"s3://{s3_bucket}/{prefix}/output/"

In [ ]:
from sagemaker.inputs import TrainingInput

train = TrainingInput(training_s3_input_location, content_type="text/csv")

In [ ]:
train.__dict__

In [ ]:
from sagemaker.image_uris import retrieve 

container = retrieve("linear-learner", region_name, "1")
container

In [ ]:
estimator = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path=training_s3_output_location,
    sagemaker_session=session)

In [ ]:
estimator.set_hyperparameters(predictor_type='regressor', mini_batch_size=4)

In [ ]:
estimator.fit({'train': train})

In [ ]:
model_data = estimator.model_data
model_data

In [ ]:
%store model_data

In [ ]:
model_uri = estimator.image_uri
model_uri

In [ ]:
%store model_uri

In [ ]:
%store X_test
%store y_test